In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d atharvadumbre/indian-sign-language-islrtc-referred

In [ ]:
!unzip /content/indian-sign-language-islrtc-referred.zip && rm /content/indian-sign-language-islrtc-referred.zip

In [11]:
import os
import cv2
import numpy as np


In [16]:
base_path = "/content/original_images"
sample_num = 300
img_size = (100,100)
classes = os.listdir(base_path)

In [18]:
def create_dataset(classes, size):
  temp = []
  size = (size, size)
  for i in classes:
    folder_path = os.path.join(base_path, i)
    img_names = os.listdir(folder_path)
    for img_name in img_names[:sample_num]:
      try:
        full_path = os.path.join(folder_path, img_name)
        img_data = cv2.imread(full_path,cv2.IMREAD_GRAYSCALE)
        img_resied = cv2.resize(img_data, size)
        temp.append([img_resied, classes.index(i)])
        print(".", end = '')
      except:
        pass
  return temp


eff_data = create_dataset(classes,100)


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [ ]:
from tensorflow import one_hot

def split_x_y(data, img_size, onehot = 0):
  x = []
  y = []
  for i in data:
    x.append(i[0])
    y.append(i[1])
  x = np.array(x)
  x = x/255.0
  x = x.reshape(-1,img_size,img_size,1)
  if onehot:
    y = one_hot(y, len(classes))

  return x,y

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x,y = split_x_y(eff_data, img_size = 100, onehot = 0)

In [ ]:
from tensorflow.keras.layers import Dense, Activation, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras import Sequential
import tensorflow_hub as hub

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [ ]:
y_test = one_hot(y_test, len(classes))
y_train = one_hot(y_train, len(classes))

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape = x.shape[1:]))
model.add(MaxPooling2D(3,3))
model.add(Activation("relu"))

model.add(Conv2D(64, (3,3)))
model.add(MaxPooling2D(3,3))
model.add(Activation("relu"))

model.add(Conv2D(64, (3,3)))
model.add(MaxPooling2D(3,3))
model.add(Activation("relu"))

model.add(Flatten())
model.add(Dense(len(classes)))
model.add(Activation("sigmoid"))

In [ ]:
model.compile(loss = "categorical_crossentropy", metrics = ["accuracy"], optimizer="adam")

In [ ]:
hists = model.fit(X_train, y_train, epochs = 10)
model.save("model1.h5")

In [42]:
from keras.models import load_model
import cv2
import numpy as np
#classes = ['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q']
classes = ['D', '3', 'T', '6', 'P', '1', '2', '7', 'G', 'W', '5', 'L', 'R', 'C', 'H', 'K', 'J', 'B', 'Y', '8', 'U', 'F', 'O', 'N', 'Z', 'E', 'V', 'X', 'A', 'S', '9', 'I', '4', 'M', '0', 'Q']
def make_pred(img_path, model_path):
  model = load_model(model_path)
  img = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
  img_resied = cv2.resize(img, (100,100))
  img_reshape = img_resied.reshape(-1, 100, 100, 1)
  res = model.predict(img_reshape)
  print(classes[np.argmax(res)])

make_pred("/content/original_images/2/1000.jpg","model1.h5")


1/1 [==============================] - 0s 93ms/step
2
